# Modèles multimodaux - CLIP

## README
Ce notebook permet la création et l'évaluation d'un modèle basé sur l'architecture existatne CLIP.

# TODO

Le chapitre 1 prépare les données nécessaires

Le chapitre 2 crée et évalue un modèle de voting simple, par moyennage des prédictions de 2 modèles (1 images et 1 texte)

Le chapitre 3 propose une version avancée, avec pondérations, apprises ou non, des résultats des deux modèles utilisés

## 1. Préparation

In [ ]:
import sys
from pathlib import Path

project_root = Path().resolve().parent
if not project_root in [Path(p).resolve() for p in sys.path]:
    sys.path.append(str(project_root))

from src import PATHS

In [ ]:
import os
import time

from src.visualization.visualize import visual_classification_report, plot_history

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LeakyReLU
from tensorflow.keras.optimizers import Adam

## 2. Chargement des données
On charge directement les embeddings sortant de CLIP

In [ ]:
features = pd.read_parquet(os.path.join(PATHS.processed_data, 'df_clip_embeddings.parquet'))

In [ ]:
data_sets = pd.read_parquet(os.path.join(PATHS.metadata, "df_data_sets.parquet"))
labels = pd.read_parquet(os.path.join(PATHS.metadata, "df_encoded_labels.parquet"))

In [ ]:
# au cas où features soit incomplet
data_sets = data_sets.join(features[[]], how="inner")
labels = labels.join(features[[]], how="inner")

features.shape, data_sets.shape, labels.shape

In [ ]:
X_train = features[data_sets.data_set == "train"]
y_train = labels[data_sets.data_set == "train"]

X_val = features[data_sets.data_set == "val"]
y_val = labels[data_sets.data_set == "val"]

X_test = features[data_sets.data_set == "test"]
y_test = labels[data_sets.data_set == "test"]

In [ ]:
del features, data_sets, labels

## 3. Création des callbacks

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',     
    factor=0.5,             
    patience=3,             
    min_lr=1e-6,            
    verbose=1              
)

checkpoint = ModelCheckpoint(
    filepath=os.path.join(PATHS.models, 'checkpoints', 'best_model.keras'),
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    verbose=1
)
os.makedirs(os.path.join(PATHS.models, 'checkpoints'), exist_ok=True)
            
early_stop = EarlyStopping(
    monitor='val_loss',       
    patience=10,
    restore_best_weights=True,
    verbose=1
)

## 4. Modèles

### 4.1. Modèle 1

In [ ]:
# Création du modèle
model = Sequential([
    Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(16, activation='softmax')
])

# Compilation
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Entrainement
t0 = time.time()
train_history = model.fit(
                    X_train,
                    y_train,
                    epochs=50,
                    batch_size=32,
                    validation_data=(X_val, y_val),
                    callbacks=[early_stop, checkpoint, reduce_lr]
                )
print(f"Entraînement réalisé en {time.time()-t0:.2f} secondes.")

# Affichage des résultats
plot_history(train_history)
visual_classification_report(model, X_test, y_test, "CLIP-based NN Model", apply_argmax_to_predict=True)

### 4.2. Modèle 2

In [ ]:
# Création du modèle
optimizer = Adam(learning_rate=1e-4)
model = Sequential([
    Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(16, activation='softmax')
])

# Compilation
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Entrainement
t0 = time.time()
train_history = model.fit(
                    X_train,
                    y_train,
                    epochs=50,
                    batch_size=32,
                    validation_data=(X_val, y_val),
                    callbacks=[early_stop, checkpoint]
                )
print(f"Entraînement réalisé en {time.time()-t0:.2f} secondes.")

# Affichage des résultats
plot_history(train_history)
visual_classification_report(model, X_test, y_test, "CLIP-based NN Model", apply_argmax_to_predict=True)

### 4.3. Modèle 3

In [ ]:
# Création du modèle
optimizer = Adam(learning_rate=1e-5)
model = Sequential([
    Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(16, activation='softmax')
])

# Compilation
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Entrainement
t0 = time.time()
train_history = model.fit(
                    X_train,
                    y_train,
                    epochs=50,
                    batch_size=32,
                    validation_data=(X_val, y_val),
                    callbacks=[early_stop, checkpoint]
                )
print(f"Entraînement réalisé en {time.time()-t0:.2f} secondes.")

# Affichage des résultats
plot_history(train_history)
visual_classification_report(model, X_test, y_test, "CLIP-based NN Model", apply_argmax_to_predict=True)

### 4.4. Modèle 4

In [ ]:
# Création du modèle
model = Sequential([
    Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(16, activation='softmax')
])

# Compilation
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Entrainement
t0 = time.time()
train_history = model.fit(
                    X_train,
                    y_train,
                    epochs=50,
                    batch_size=32,
                    validation_data=(X_val, y_val),
                    callbacks=[reduce_lr, early_stop, checkpoint]
                )
print(f"Entraînement réalisé en {time.time()-t0:.2f} secondes.")

# Affichage des résultats
plot_history(train_history)
visual_classification_report(model, X_test, y_test, "CLIP-based NN Model", apply_argmax_to_predict=True)

### 4.5. Modèle 5

In [ ]:
# Création du modèle
optimizer = Adam(learning_rate=1e-4)
model = Sequential([
    Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(16, activation='softmax')
])

# Compilation
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Entrainement
t0 = time.time()
train_history = model.fit(
                    X_train,
                    y_train,
                    epochs=50,
                    batch_size=32,
                    validation_data=(X_val, y_val),
                    callbacks=[reduce_lr, early_stop, checkpoint]
                )
print(f"Entraînement réalisé en {time.time()-t0:.2f} secondes.")

# Affichage des résultats
plot_history(train_history)
visual_classification_report(model, X_test, y_test, "CLIP-based NN Model", apply_argmax_to_predict=True)

### 4.6. Modèle 6

In [ ]:
# Création du modèle
optimizer = Adam(learning_rate=1e-4)
model = Sequential([
    Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.4),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(16, activation='softmax')  # 16 classes
])

# Compilation
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Entrainement
t0 = time.time()
train_history = model.fit(
                    X_train,
                    y_train,
                    epochs=50,
                    batch_size=32,
                    validation_data=(X_val, y_val),
                    callbacks=[reduce_lr, early_stop, checkpoint]
                )
print(f"Entraînement réalisé en {time.time()-t0:.2f} secondes.")

# Affichage des résultats
plot_history(train_history)
visual_classification_report(model, X_test, y_test, "CLIP-based NN Model", apply_argmax_to_predict=True)

### 4.7. Modèle 7

In [ ]:
# Création du modèle
optimizer = Adam(learning_rate=1e-4)
model = Sequential([
    Dense(512, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(16, activation='softmax')
])

# Compilation
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Entrainement
t0 = time.time()
train_history = model.fit(
                    X_train,
                    y_train,
                    epochs=50,
                    batch_size=32,
                    validation_data=(X_val, y_val),
                    callbacks=[reduce_lr, early_stop, checkpoint]
                )
print(f"Entraînement réalisé en {time.time()-t0:.2f} secondes.")

# Affichage des résultats
plot_history(train_history)
visual_classification_report(model, X_test, y_test, "CLIP-based NN Model", apply_argmax_to_predict=True)

### 4.8. Modèle 8

In [ ]:
# Création du modèle
optimizer = Adam(learning_rate=1e-4)
model = Sequential([
    Dense(256, input_shape=(X_train.shape[1],)),
    LeakyReLU(alpha=0.1),
    Dense(128),
    LeakyReLU(alpha=0.1),
    Dense(64),
    LeakyReLU(alpha=0.1),
    Dense(16, activation='softmax')  # 16 classes
])

# Compilation
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Entrainement
t0 = time.time()
train_history = model.fit(
                    X_train,
                    y_train,
                    epochs=50,
                    batch_size=32,
                    validation_data=(X_val, y_val),
                    callbacks=[reduce_lr, early_stop, checkpoint]
                )
print(f"Entraînement réalisé en {time.time()-t0:.2f} secondes.")

# Affichage des résultats
plot_history(train_history)
visual_classification_report(model, X_test, y_test, "CLIP-based NN Model", apply_argmax_to_predict=True)

### 4.9. Modèle 9

In [ ]:
# Création du modèle
optimizer = Adam(learning_rate=1e-4)
model = Sequential([
    Dense(256, input_shape=(X_train.shape[1],)),
    Dropout(0.4),
    LeakyReLU(alpha=0.1),
    Dense(128),
    Dropout(0.2),
    LeakyReLU(alpha=0.1),
    Dense(128),
    Dropout(0.2),
    LeakyReLU(alpha=0.1),
    Dense(64),
    Dropout(0.1),
    LeakyReLU(alpha=0.1),
    Dense(64),
    Dropout(0,1),
    LeakyReLU(alpha=0.1),
    Dense(16, activation='softmax')  # 16 classes
])

# Compilation
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Entrainement
t0 = time.time()
train_history = model.fit(
                    X_train,
                    y_train,
                    epochs=50,
                    batch_size=32,
                    validation_data=(X_val, y_val),
                    callbacks=[reduce_lr, early_stop, checkpoint]
                )
print(f"Entraînement réalisé en {time.time()-t0:.2f} secondes.")

# Affichage des résultats
plot_history(train_history)
visual_classification_report(model, X_test, y_test, "CLIP-based NN Model", apply_argmax_to_predict=True)

### 4.10. Modèle 10

In [ ]:
# Création du modèle
optimizer = Adam(learning_rate=1e-4)
model = Sequential([
    Dense(64, input_shape=(X_train.shape[1],)),
    LeakyReLU(alpha=0.1),
    Dense(32),
    LeakyReLU(alpha=0.1),
    Dense(32),
    LeakyReLU(alpha=0.1),
    Dense(16, activation='softmax')  # 16 classes
])

# Compilation
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Entrainement
t0 = time.time()
train_history = model.fit(
                    X_train,
                    y_train,
                    epochs=50,
                    batch_size=32,
                    validation_data=(X_val, y_val),
                    callbacks=[reduce_lr, early_stop, checkpoint]
                )
print(f"Entraînement réalisé en {time.time()-t0:.2f} secondes.")

# Affichage des résultats
plot_history(train_history)
visual_classification_report(model, X_test, y_test, "CLIP-based NN Model", apply_argmax_to_predict=True)

### 4.11. Modèle 11

In [ ]:
# Création du modèle
optimizer = Adam(learning_rate=1e-4)
model = Sequential([
    Dense(512, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.4),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(16, activation='softmax')  # 16 classes
])

# Compilation
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Entrainement
t0 = time.time()
train_history = model.fit(
                    X_train,
                    y_train,
                    epochs=50,
                    batch_size=32,
                    validation_data=(X_val, y_val),
                    callbacks=[reduce_lr, early_stop, checkpoint]
                )
print(f"Entraînement réalisé en {time.time()-t0:.2f} secondes.")

# Affichage des résultats
plot_history(train_history)
visual_classification_report(model, X_test, y_test, "CLIP-based NN Model", apply_argmax_to_predict=True)

### 4.12. Modèle 12

In [ ]:
# Création du modèle
optimizer = Adam(learning_rate=1e-4)
model = Sequential([
    Dense(512, input_shape=(X_train.shape[1],)),
    LeakyReLU(alpha=0.1),
    Dropout(0.4),
    Dense(128),
    LeakyReLU(alpha=0.1),
    Dropout(0.3),
    Dense(32),
    LeakyReLU(alpha=0.1),
    Dropout(0.2),
    Dense(16, activation='softmax')  # 16 classes
])

# Compilation
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Entrainement
t0 = time.time()
train_history = model.fit(
                    X_train,
                    y_train,
                    epochs=50,
                    batch_size=32,
                    validation_data=(X_val, y_val),
                    callbacks=[reduce_lr, early_stop, checkpoint]
                )
print(f"Entraînement réalisé en {time.time()-t0:.2f} secondes.")

# Affichage des résultats
plot_history(train_history)
visual_classification_report(model, X_test, y_test, "CLIP-based NN Model", apply_argmax_to_predict=True)

### 4.13. Modèle 13

In [ ]:
model = Sequential([
    Dense(512, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.4),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(16, activation='softmax')  # 16 classes
])

# Compilation avec sparse_categorical_crossentropy
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

train_history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=32,
    callbacks=[reduce_lr]
)

In [ ]:
plot_history(train_history)

In [ ]:
visual_classification_report(model, X_test, y_test, "CLIP-based NN Model")

In [ ]:
from tensorflow.keras.layers import Dense, LeakyReLU
model = Sequential([
    Dense(512, input_shape=(X_train.shape[1],)),
    LeakyReLU(alpha=0.1),
    Dropout(0.4),
    Dense(128),
    LeakyReLU(alpha=0.1),
    Dropout(0.3),
    Dense(32),
    LeakyReLU(alpha=0.1),
    Dropout(0.2),
    Dense(16, activation='softmax')  # 16 classes
])

# Compilation avec sparse_categorical_crossentropy
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])



train_history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=32,
    callbacks=[reduce_lr]
)

In [ ]:
plot_history(train_history)

In [ ]:
visual_classification_report(model, X_test, y_test, "CLIP-based NN Model")